In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks/pytorch-forecasting-master')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

!pip install tensorflow
!pip install pytorch_lightning
!pip install optuna
!pip install plotly

Mounted at /content/drive
Fri Mar  1 03:42:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

In [2]:
#from ctypes import FormatError
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import os,sys

import torch
# same thing as setting seed of random generator (experiments are reproducible, loss curves will be same etc, train in the same order)
# to see if modification makes any improvement
torch.use_deterministic_algorithms(True)

from pytorch_forecasting.data.encoders import TorchNormalizer # normalize inputs (inputs is tsfresh features)
from pytorch_forecasting.metrics import SMAPE, RMSE # this one uses cpu
from torchmetrics import R2Score, SymmetricMeanAbsolutePercentageError, MeanSquaredError # this one uses gpu
from torchmetrics.regression import WeightedMeanAbsolutePercentageError # (use this also)

import matplotlib.pyplot as plt
import pandas as pd
from pytorch_forecasting.data import TimeSeriesDataSet
from pytorch_forecasting.data import NaNLabelEncoder
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor # running in the background (continuously called)
import pytorch_lightning as pl
import torch
from pytorch_forecasting.data.encoders import TorchNormalizer
import os,sys
import numpy as np
from statsmodels.tsa.stattools import adfuller #adaptive dickie fuller test
from statsmodels.tsa.seasonal import seasonal_decompose # trend seasonality residual
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import acf,pacf
from scipy.signal import find_peaks
import operator
import statsmodels.api as sm
from itertools import combinations
import pickle
from pytorch_forecasting import Baseline # naive forecasting
import random
from pytorch_forecasting import DeepAR,NegativeBinomialDistributionLoss
from itertools import product
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna
from optuna.trial import TrialState
import plotly
# from optuna.visualization import plot_parallel_coordinate, plot_optimization_history, plot_slice, plot_param_importances
# from IPython.display import display

In [3]:

"""
Set Random seed
"""

random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
## additional seeding to ensure reproduciblility.
pl.seed_everything(0)

import os

INFO:lightning_fabric.utilities.seed:Seed set to 0


In [4]:
encoder_len = 8 # looks back 8 hours in the past for prediction
pred_len = 1 # 1, 6, 12 # how many timeseteps you want to predict in the future (forecasting), 1 means present (nowcasting)
#cov_lag_len= pred_len

# not using covariates yet, so put as 0
cov_lag_len= 0 #we can use forecasted values, even for inflow

In [5]:
"""
Import pre-processed Data (tsfresh features)

response and target are the same thing
"""

### Your code
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks')
############ Basically df = raw_feat_df <-------------------------
df = pd.read_parquet('[Draft_2]_combined_dd_ss_raw_feat_data_significant_lags_pruned.parquet', engine='pyarrow')
df
os.chdir('/content/drive/MyDrive/FYP_Python_Notebooks/pytorch-forecasting-master')

In [6]:
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
0,167,2021-01-07 23:00:00,4,0.0,10.0,8.0,1.0,1.0,5.0,3.0,...,0.0,-0.517121,0.028399,-0.517109,-0.362827,-0.207694,-6.0,-0.000421,0.0,0.574259
1,168,2021-01-08 00:00:00,4,1.0,4.0,13.0,0.0,2.0,0.0,1.0,...,1.0,-1.000000,0.028399,-0.517109,-0.362827,-0.207694,-3.0,-0.000421,1.0,0.574259
2,169,2021-01-08 01:00:00,4,2.0,1.0,9.0,0.0,0.0,1.0,2.0,...,2.0,-0.250000,-1.000000,-0.517109,-0.362827,-0.207694,0.0,-0.000421,1.0,0.574259
3,170,2021-01-08 02:00:00,4,0.0,1.0,5.0,0.0,1.0,2.0,3.0,...,3.0,-0.146199,-0.438596,-0.146199,-0.362827,-0.207694,2.0,-0.684211,1.0,3.750693
4,171,2021-01-08 03:00:00,4,0.0,3.0,2.0,1.0,1.0,0.0,0.0,...,3.0,-0.131773,-0.532020,-0.131773,-0.362827,-0.207694,-3.0,-0.919951,2.0,-3.126259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631191,24808,2023-10-31 16:00:00,263,126.0,176.0,136.0,23.0,226.0,142.0,113.0,...,14.0,-0.421845,-0.103924,-0.421845,-0.342896,0.203789,-64.0,0.351349,11.0,0.235154
1631192,24809,2023-10-31 17:00:00,263,112.0,153.0,144.0,9.0,185.0,126.0,82.0,...,13.0,-0.546637,0.012524,-0.546637,-0.408289,-0.018269,10.0,0.202870,12.0,0.405906
1631193,24810,2023-10-31 18:00:00,263,99.0,135.0,94.0,7.0,136.0,112.0,77.0,...,14.0,-0.540946,0.001364,-0.540946,-0.411744,-0.030810,14.0,0.203590,11.0,0.406788
1631194,24811,2023-10-31 19:00:00,263,60.0,119.0,62.0,4.0,85.0,99.0,30.0,...,14.0,-0.443736,-0.035146,-0.443736,-0.288941,0.078741,-8.0,0.206272,11.0,0.743705


In [7]:
first_idx = df['time_idx'][0]
total_time_idx = df[df.group == 4].shape[0]
reduced_df_time_idx = [*range(total_time_idx-int(0.01*total_time_idx), total_time_idx+1, 1)]
df = df[df['time_idx'].isin(reduced_df_time_idx)]

In [8]:
df

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
24233,24400,2023-10-14 16:00:00,4,14.0,33.0,32.0,23.0,35.0,12.0,5.0,...,14.0,-0.184814,-0.402034,-0.184814,-0.451616,-0.220442,3.0,0.021835,11.0,0.245327
24234,24401,2023-10-14 17:00:00,4,4.0,40.0,48.0,22.0,40.0,14.0,10.0,...,13.0,-0.172869,-0.377358,-0.172869,-0.419787,-0.155920,-17.0,0.040581,12.0,-0.007865
24235,24402,2023-10-14 18:00:00,4,9.0,33.0,40.0,21.0,57.0,4.0,8.0,...,14.0,-0.208144,-0.292158,-0.208144,-0.350674,-0.056880,21.0,0.107403,11.0,0.098075
24236,24403,2023-10-14 19:00:00,4,10.0,42.0,39.0,11.0,67.0,9.0,22.0,...,13.0,-0.270866,-0.267799,-0.270866,-0.368179,-0.138159,4.0,0.088303,12.0,-0.023266
24237,24404,2023-10-14 20:00:00,4,41.0,41.0,59.0,7.0,51.0,10.0,45.0,...,13.0,-0.332988,-0.140534,-0.332988,-0.282769,-0.097979,14.0,0.074037,12.0,-0.235285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631025,24642,2023-10-24 18:00:00,263,110.0,226.0,90.0,3.0,158.0,93.0,57.0,...,16.0,-0.411228,0.184453,-0.411228,0.018468,-0.092993,-22.0,-0.159287,9.0,0.204759
1631026,24643,2023-10-24 19:00:00,263,64.0,185.0,57.0,4.0,77.0,110.0,18.0,...,15.0,-0.374424,0.172698,-0.374424,0.037804,-0.067731,-33.0,-0.134525,10.0,0.141492
1631027,24644,2023-10-24 20:00:00,263,38.0,136.0,24.0,0.0,41.0,64.0,8.0,...,15.0,-0.361882,0.189081,-0.361882,0.066881,-0.007411,-1.0,-0.094464,10.0,0.680821
1631028,24645,2023-10-24 21:00:00,263,21.0,85.0,13.0,5.0,21.0,38.0,7.0,...,15.0,-0.368928,0.159361,-0.368928,0.026916,-0.078509,-10.0,-0.134914,10.0,0.744708


In [9]:
check = df[df['group'] == 4]
check

,time_idx,datetime,group,target,inflow_lag_0,inflow_lag_1,inflow_lag_2,inflow_lag_3,outflow_lag_0,outflow_lag_1,...,inflow_diff2_per1__count_above_mean,inflow_diff2_per1__autocorrelation__lag_1,inflow_diff2_per1__autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_1,inflow_diff2_per1__partial_autocorrelation__lag_2,inflow_diff2_per1__partial_autocorrelation__lag_3,"inflow_diff2_per1__fft_coefficient__attr_""real""__coeff_0",inflow_diff2_per1__autocorrelation__lag_3,inflow_diff2_per1__count_below_mean,inflow_diff2_per1__kurtosis
24233,24400,2023-10-14 16:00:00,4,14.0,33.0,32.0,23.0,35.0,12.0,5.0,...,14.0,-0.184814,-0.402034,-0.184814,-0.451616,-0.220442,3.0,0.021835,11.0,0.245327
24234,24401,2023-10-14 17:00:00,4,4.0,40.0,48.0,22.0,40.0,14.0,10.0,...,13.0,-0.172869,-0.377358,-0.172869,-0.419787,-0.155920,-17.0,0.040581,12.0,-0.007865
24235,24402,2023-10-14 18:00:00,4,9.0,33.0,40.0,21.0,57.0,4.0,8.0,...,14.0,-0.208144,-0.292158,-0.208144,-0.350674,-0.056880,21.0,0.107403,11.0,0.098075
24236,24403,2023-10-14 19:00:00,4,10.0,42.0,39.0,11.0,67.0,9.0,22.0,...,13.0,-0.270866,-0.267799,-0.270866,-0.368179,-0.138159,4.0,0.088303,12.0,-0.023266
24237,24404,2023-10-14 20:00:00,4,41.0,41.0,59.0,7.0,51.0,10.0,45.0,...,13.0,-0.332988,-0.140534,-0.332988,-0.282769,-0.097979,14.0,0.074037,12.0,-0.235285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24475,24642,2023-10-24 18:00:00,4,2.0,37.0,28.0,25.0,15.0,2.0,1.0,...,14.0,-0.413482,-0.050009,-0.413482,-0.266547,0.201066,-7.0,0.291249,11.0,1.463756
24476,24643,2023-10-24 19:00:00,4,7.0,24.0,24.0,15.0,8.0,2.0,2.0,...,12.0,-0.499592,0.091354,-0.499592,-0.210869,0.093410,18.0,0.117064,13.0,5.897417
24477,24644,2023-10-24 20:00:00,4,5.0,45.0,5.0,7.0,1.0,7.0,3.0,...,14.0,-0.627924,0.272168,-0.627924,-0.201616,0.074929,-21.0,-0.035217,11.0,6.742545
24478,24645,2023-10-24 21:00:00,4,2.0,19.0,5.0,2.0,1.0,5.0,3.0,...,14.0,-0.790665,0.418261,-0.790665,-0.551926,-0.264828,-1.0,-0.145871,11.0,5.264853


In [10]:
numeric_cols = list(df.columns)
numeric_cols = [e for e in numeric_cols if e not in ('time_idx', 'datetime', 'group', 'target')]
print(numeric_cols)

['inflow_lag_0', 'inflow_lag_1', 'inflow_lag_2', 'inflow_lag_3', 'outflow_lag_0', 'outflow_lag_1', 'outflow_lag_2', 'outflow_lag_3', 'inflow', 'inflow_diff1_per1', 'inflow_diff1_per2', 'inflow_diff2_per1', 'inflow_lag_0__fft_coefficient__attr_"real"__coeff_0', 'inflow_lag_0__fft_coefficient__attr_"real"__coeff_1', 'inflow_lag_0__skewness', 'inflow_lag_0__kurtosis', 'inflow_lag_0__count_above_mean', 'inflow_lag_0__count_below_mean', 'inflow_lag_0__autocorrelation__lag_1', 'inflow_lag_0__autocorrelation__lag_2', 'inflow_lag_0__autocorrelation__lag_3', 'inflow_lag_0__partial_autocorrelation__lag_1', 'inflow_lag_0__partial_autocorrelation__lag_2', 'inflow_lag_0__partial_autocorrelation__lag_3', 'inflow_lag_1__fft_coefficient__attr_"real"__coeff_0', 'inflow_lag_1__fft_coefficient__attr_"real"__coeff_1', 'inflow_lag_1__skewness', 'inflow_lag_1__kurtosis', 'inflow_lag_1__count_above_mean', 'inflow_lag_1__count_below_mean', 'inflow_lag_1__autocorrelation__lag_1', 'inflow_lag_1__autocorrelation

In [11]:
#################### add date information ts ####################

# create datetime features from datetime column in raw_feat_df
df['_hour_of_day'] = df["datetime"].dt.hour.astype(str)
df['_day_of_week'] = df["datetime"].dt.dayofweek.astype(str)
df['_day_of_month'] = df["datetime"].dt.day.astype(str)
df['_day_of_year'] = df["datetime"].dt.dayofyear.astype(str)
df['_week_of_year'] = df["datetime"].dt.weekofyear.astype(str)
df['_month_of_year'] = df["datetime"].dt.month.astype(str)
df['_year'] = df["datetime"].dt.year.astype(str)
#################### add date information ts ####################

hour_of_day_nunique = df['_hour_of_day'].nunique()
day_of_week_nunique = df['_day_of_week'].nunique()
day_of_month_nunique = df['_day_of_month'].nunique()
day_of_year_nunique = df['_day_of_year'].nunique()
week_of_year_nunique = df['_week_of_year'].nunique()
month_of_year_nunique = df['_month_of_year'].nunique()
year_nunique = df['_year'].nunique()

In [12]:
print(hour_of_day_nunique)
print(day_of_week_nunique)
print(day_of_month_nunique)
print(day_of_year_nunique)
print(week_of_year_nunique)
print(month_of_year_nunique)
print(year_nunique)

24
7
11
11
3
1
1


In [13]:
####### train val test split of tsfresh dataframe #########

### you want a the idx for each train, validation and test

### train = 0.7 of each zone (eg. time idx 167 to roughly 20000) eg train = 0.7 * (24812-167)
### validation = 0.2 of each zone (eg. time idx 20000 to 22000)
### test = 0.1 of each zone (eg. time idx 22000 to 24000)

Target = 'target'

total_time_idx = df[df.group == 4].shape[0]

first_idx = df.loc[df['group'] == 4, 'time_idx'].iloc[0]

tr_stop_idx = int(0.7*total_time_idx)
val_start_idx = tr_stop_idx - encoder_len
val_stop_idx = val_start_idx + int(0.2*total_time_idx) # use (tr_stop_idx - encoder_len) as encoder from train set for validatoin
tes_start_idx = val_stop_idx - encoder_len
tes_stop_idx = total_time_idx-1

print(tr_stop_idx)
print(val_start_idx)
print(val_stop_idx)
print(tes_start_idx)
print(tes_stop_idx)

172
164
213
205
246


In [14]:
train_time_idx = [*range(first_idx, first_idx+tr_stop_idx+1, 1)]
print(train_time_idx[0], train_time_idx[-1])
val_time_idx = [*range(first_idx+val_start_idx, first_idx+val_stop_idx+1, 1)]
print(val_time_idx[0], val_time_idx[-1])
test_time_idx = [*range(first_idx+tes_start_idx, first_idx+tes_stop_idx+1, 1)]
print(test_time_idx[0], test_time_idx[-1])

24400 24572
24564 24613
24605 24646


In [15]:
unique_zones = list(df['group'].unique())

In [16]:
"""
set inputs here
(hyperparameters grid search)

"""
######### Network Architecture ###################
p = 10 # patience no. of epochs

Loss=NegativeBinomialDistributionLoss() # because we assume that the demand follows a negative bionmial distribution

######### Network Architecture ###################


######### Training Routine ###################
fdv_steps = 10 # fast_dev_run -> small set of training to see if training is happening correctly (one of the params in deepar model)
######### Training Routine ###################


# this is for naive forecast, getting past week's demand and predicting purely based on that
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################
season_len = 168 # length of season ()
num_past_seas = 6 # number of past seasons to use in averaging (look at 6 week's back, getting average of these 6 values)
#seas_pred_strt_idx = 2035 # seasonal naive forecast start index, in hours use the df dataframe
############## Inputs for 2) Persistance model ( seasonal naive forecast ) #######################


In [17]:
import os

os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # or ':16:8' depending on your preference

In [22]:
"""
Full Training Routine
with hyperparmeter grid search

Load data into TimeSeriesDataSet object

for fast development run
uncomment fast_dev_run = fdv_steps

"""

# early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=p, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

# param_comb_cnt=-1
#for neu,lay,bat,lr,enc_len,pred_len,drop,cov_pair,num_ep in product(*[x for x in hparams_grid.values()]):

def objective(trial): #optuna objective function to find best hyperparameters
# minimize the average validation loss

  # optuna tries to find best combination given this hyperparameter searchspace

  # first start broad
  neu = trial.suggest_int(name="neu",low=10,high=1000,step=100,log=False) # number of neurons in lstm cell
  lay = trial.suggest_int(name="lay",low=1,high=10,step=2,log=False) # number of lstm layers
  bat = trial.suggest_int(name="bat",low=32,high=1024,step=32,log=False) # batch size
  lr = trial.suggest_float(name="lr",low=0.00001,high=0.1,log=True) #learning rate, log does not have step
  drop = trial.suggest_float(name="dropout",low=0,high=0.8,step=0.2,log=False) # dont use some of the cells to distribute learning across the cells

  num_cols_list = numeric_cols
  cat_dict = {"_hour_of_day": NaNLabelEncoder(add_nan=True).fit(df._hour_of_day),
          "_day_of_week": NaNLabelEncoder(add_nan=True).fit(df._day_of_week), "_day_of_month" : NaNLabelEncoder(add_nan=True).fit(df._day_of_month), "_day_of_year" : NaNLabelEncoder(add_nan=True).fit(df._day_of_year),
              "_week_of_year": NaNLabelEncoder(add_nan=True).fit(df._week_of_year), "_month_of_year": NaNLabelEncoder(add_nan=True).fit(df._month_of_year) ,"_year": NaNLabelEncoder(add_nan=True).fit(df._year)}
  cat_list = ["_hour_of_day","_day_of_week","_day_of_month","_day_of_year","_week_of_year","_month_of_year","_year"]
  embed_size_dict = {'_hour_of_day':(hour_of_day_nunique+1,5),'_day_of_week':(day_of_week_nunique+1,5),'_day_of_month':(day_of_month_nunique+1,5), '_day_of_year':(day_of_year_nunique+1,8), '_week_of_year':(week_of_year_nunique+1,3), '_month_of_year':(month_of_year_nunique+1,3),'_year':(year_nunique+1,1)}
  # ^ converting hour into a 5 dimensional vector, etc etc, day of year = number of unique values in this column + 1, same for week of year, month of year and year

  # for num_ep in range(20, 30, 3):
    # trial.set_user_attr("num_ep", num_ep)

  random_zones = random.sample(unique_zones, 5)

  early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=True,
    mode='min'
  )

  train_dataset = TimeSeriesDataSet(
    df[(df['time_idx'].isin(train_time_idx)) & (df['group'].isin(random_zones))],
    time_idx="time_idx",
    target=Target,
    categorical_encoders=cat_dict,
    group_ids=["group"],
    min_encoder_length=encoder_len,
    max_encoder_length=encoder_len,
    min_prediction_length=pred_len,
    max_prediction_length=pred_len,
    time_varying_unknown_reals=[Target],
    time_varying_known_reals=num_cols_list,
    time_varying_known_categoricals=cat_list,
    add_relative_time_idx=False,
    randomize_length=False,
    scalers={},
    target_normalizer=TorchNormalizer(method="identity",center=False,transformation=None)
  )

  val_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[(df['time_idx'].isin(val_time_idx)) & (df['group'].isin(random_zones))], stop_randomization=True, predict=False)
  # test_dataset = TimeSeriesDataSet.from_dataset(train_dataset,df[df['time_idx'].isin(test_time_idx)], stop_randomization=True)

  train_dataloader = train_dataset.to_dataloader(train=True, batch_size=bat)
  val_dataloader = val_dataset.to_dataloader(train=False, batch_size=bat)
  # test_dataloader = test_dataset.to_dataloader(train=False, batch_size=bat)
  """
  Machine Learning predictions START
  1) DeepAR

  """
  trainer = pl.Trainer(
      max_epochs=10,
      accelerator="auto",
      gradient_clip_val=0.1,
      limit_train_batches=1.0,
      limit_val_batches=1.0,
      # fast_dev_run=fdv_steps,
      logger=True,
      callbacks=[lr_logger, early_stop_callback]    #, early_stop_callback]
  )


  #print(f"training routing:\n \n {trainer}")
  deepar = DeepAR.from_dataset(
      train_dataset,
      learning_rate=lr,
      hidden_size=neu,
      rnn_layers=lay,
      dropout=drop,
      loss=Loss,
      log_interval=20,
      log_val_interval=6,
      log_gradient_flow=False,
      embedding_sizes=embed_size_dict
      # reduce_on_plateau_patience=3,
  )

  #print(f"Number of parameters in network: {deepar.size()/1e3:.1f}k")
  # print(f"Model :\n \n {deepar}")
  torch.set_num_threads(10)
  trainer.fit(
      deepar,
      train_dataloaders=train_dataloader,
      val_dataloaders=val_dataloader,
  )

  # val_rmse = trainer.callback_metrics['val_RMSE'].item()
  val_wmape = trainer.callback_metrics['val_WMAPE'].item()
  val_rmse = trainer.callback_metrics['val_RMSE'].item()

  # trial.report(val_wmape, 10)
  # trial.report(val_rmse, 10)

  # Handle pruning based on the intermediate value.
  # if trial.should_prune():
  #     raise optuna.exceptions.TrialPruned()

  return val_wmape, val_rmse #optuna is trying to minimise this <---


########## optuna results #####################
if __name__ == "__main__":

  study = optuna.create_study(directions=["minimize", "minimize"])
  study.optimize(objective, n_trials=10) # can specify the number of trials to a number less than 960 for example, eg can set n_trials = 100

  pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

  print("Study statistics: ")
  print("  Number of finished trials: ", len(study.trials))
  print("  Number of pruned trials: ", len(pruned_trials))
  print("  Number of complete trials: ", len(complete_trials))

  print("Best trial:")
  trial = study.best_trials
  # best_num_ep = study.trials[trial.number].user_attrs['num_ep']

  print("  Value: ", trial.value)

  print("  Params: ")
  for key, value in trial.params.items():
      print("    {}: {}".format(key, value))
########## optuna results #####################

[I 2024-03-01 03:56:03,531] A new study created in memory with name: no-name-18f8e185-c6e2-429a-b4fd-caf064ade5e0
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                   | Type                             | Params
----------------------------------------------------------------------------
0 | loss     

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

-------------------------------------- prediction is here--->  tensor([[[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7012]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [[0.7108, 0.7013]],

        [

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.callbacks.early_stopping:Metric val_loss improved by 0.007 >= min_delta = 0. New best score: 3.344


tensor([[21.],
        [40.],
        [26.],
        [35.],
        [34.],
        [27.],
        [31.],
        [34.],
        [13.],
        [24.],
        [20.],
        [ 8.],
        [ 4.],
        [ 5.],
        [ 1.],
        [ 0.],
        [ 0.],
        [ 1.],
        [ 5.],
        [10.],
        [ 9.],
        [16.],
        [18.],
        [27.],
        [25.],
        [19.],
        [24.],
        [19.],
        [28.],
        [23.],
        [ 0.],
        [ 0.]], device='cuda:0') 

-------------------------------------- predictions_squeezed is here--->  tensor([[24.2575],
        [24.2581],
        [24.2585],
        [24.2593],
        [24.2598],
        [24.2609],
        [24.2613],
        [24.2618],
        [24.2618],
        [24.2620],
        [24.2618],
        [24.2618],
        [24.2616],
        [24.2612],
        [24.2608],
        [24.2603],
        [24.2590],
        [24.2587],
        [24.2574],
        [24.2545],
        [24.2525],
        [24.2536],
        [

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
[I 2024-03-01 04:00:10,638] Trial 9 finished with values: [1.065673589706421, 16.562076568603516] and parameters: {'neu': 510, 'lay': 5, 'bat': 32, 'lr': 0.009961608565422475, 'dropout': 0.0}. 


Study statistics: 
  Number of finished trials:  10
  Number of pruned trials:  0
  Number of complete trials:  10
Best trial:


AttributeError: 'list' object has no attribute 'value'

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

In [ ]:
from IPython.display import display
fig = optuna.visualization.plot_parallel_coordinate(study, target=lambda t: t.values[0], target_name="val_WMAPE")
display(fig)
fig = optuna.visualization.plot_parallel_coordinate(study, target=lambda t: t.values[0], target_name="val_RMSE")
display(fig)
fig = optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[0], target_name="val_WMAPE")
display(fig)
fig = optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[0], target_name="val_RMSE")
display(fig)
fig = optuna.visualization.plot_slice(study, target=lambda t: t.values[0], target_name="val_WMAPE")
display(fig)
fig = optuna.visualization.plot_slice(study, target=lambda t: t.values[0], target_name="val_RMSE")
display(fig)
fig = optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0], target_name="val_WMAPE")
display(fig)
fig = optuna.visualization.plot_param_importances(study, target=lambda t: t.values[0], target_name="val_RMSE")
display(fig)

In [ ]:
fig = optuna.visualization.plot_pareto_front(study, target_names=["val_WMAPE", "val_RMSE"])
display(fig)